# T0. PREPARE POINT CLOUDS

Import and prepare point clouds for semantic segmentation, instance segmentation, etc.
To run these scripts, create a python 3.10 environment & install geomapi (numpy, opend3d, ifcopenshell, trimesh, ...)

## LIBRARIES

In [22]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import time
import json
from scipy.spatial.transform import Rotation   
import copy
from datetime import datetime

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
from geomapi.tools import progresstools as pt

import geomapi.tools as tl

#import utils
from context import utils
import utils as utl
import utils.t1_utils as t1


In [23]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
%autoreload 2

## INPUTS

In [25]:
start_time = time.time()

In [44]:
name='beton'

path=Path(os.getcwd()).parents[2]/'data'
# pcd_input_path=os.path.join(path,f'{name}.laz')
pcd_input_folder=os.path.join(path,'beton') 
class_file=path/'_classes.json'
pcd_output_path= os.path.join(path,f'{name}_labels.laz') 
objects_output_path=os.path.join(path,f'{name}_objects.json') 

distance_threshold=0.1 #! transfer distance threshold


Import PCD

In [45]:
pcdNodes=[]
for f in [l for l in ut.get_list_of_files(pcd_input_folder) if l.endswith('.las')]: # cluster the remainder
    
    #read pcd
    print(f)
    las=laspy.read(f)
    
    #split into components
    if 'Original cloud index' in list(las.point_format.dimension_names):
        pcds,_=gmu.split_pcd_by_labels(gmu.las_to_pcd(las),las['Original cloud index'])   #object
    else:
        pcds= [gmu.las_to_pcd(las)]
    
    #create nodes
    for p in pcds:
        pcdNodes.append(PointCloudNode(resource=p,
                                        name=ut.get_filename(f),
                                        class_id=255,
                                        object_id=0,
                                        color=ut.random_color()))
print(list(las.point_format.dimension_names))
print(f'{len(pcdNodes)} pcdNodes created!')

c:/Users/Maarten/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/beton/-1_rest.las
c:/Users/Maarten/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/beton/0_Floors.las
c:/Users/Maarten/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/beton/1_Ceilings.las
c:/Users/Maarten/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/beton/2_Walls.las
c:/Users/Maarten/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/beton/3_Columns.las
c:/Users/Maarten/OneDrive - KU Leuven/2024-05 CVPR scan-to-BIM challenge/data/beton/4_doors.las
['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'red', 'green', 'blue', 'Original cloud index']
85 pcdNodes created!


Import Classes

In [46]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': -1, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 0, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 1, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 2, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 3, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 4, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 5, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


## PROCESSING

Add segmentation classes to pcdNodes

In [47]:
#for every pcdNode, assign class_id and material_id (if present)
class_names = [class_obj['name'] for class_obj in json_data['classes']]
for i,n in enumerate(pcdNodes):
    class_id=float(n.name.split('_')[0])
 
    #select segmantation class
    class_obj=next((class_obj for class_obj in json_data['classes'] if float(class_obj['id']) ==class_id), json_data['classes'][0])
    n.class_id=class_obj['id']
    n.object_id=i


In [48]:
{key:value for key, value in pcdNodes[0].__dict__.items() if not key.startswith('__') and not callable(key)}              

{'_e57Index': 0,
 'pointCount': 916889,
 'e57XmlPath': None,
 '_cartesianBounds': array([1.03772125e+05, 1.03798594e+05, 1.94649829e+05, 1.94676734e+05,
        8.37500000e+00, 2.03540000e+01]),
 '_orientedBounds': array([[1.03788172e+05, 1.94646832e+05, 7.31253558e+00],
        [1.03801147e+05, 1.94668996e+05, 6.82782489e+00],
        [1.03769160e+05, 1.94658008e+05, 9.38837145e+00],
        [1.03789291e+05, 1.94646447e+05, 1.96392990e+01],
        [1.03783255e+05, 1.94679786e+05, 2.12304242e+01],
        [1.03770279e+05, 1.94657622e+05, 2.17151349e+01],
        [1.03802267e+05, 1.94668611e+05, 1.91545883e+01],
        [1.03782136e+05, 1.94680172e+05, 8.90366075e+00]]),
 '_orientedBoundingBox': OrientedBoundingBox: center: (103786, 194663, 14.2715), extent: 25.6874, 22.1506, 12.3835),
 '_subject': rdflib.term.URIRef('file:///-1_rest'),
 '_graph': None,
 '_graphPath': None,
 '_path': None,
 '_name': '-1_rest',
 '_timestamp': None,
 '_resource': PointCloud with 916889 points.,
 '_cartes

In [31]:
# #visualize
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in pcdNodes])
# o3d.visualization.draw_geometries([joined_pcd])

Compute 3D objects

In [49]:
#extract parameters for each total_pcd_nodes
for n in pcdNodes:
    
    #take points
    points=np.asarray(n.resource.points)
    
    #project points in 2D
    points2d=points[:,0:2]
    
    #select lowest and highest point
    zmin=np.min(points[:,2])
    zmax=np.max(points[:,2])
    
    #translate data
    new_points=np.vstack((np.hstack((points2d,np.full((points2d.shape[0],1),zmin))),
                          np.hstack((points2d,np.full((points2d.shape[0],1),zmax+50)))))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(new_points)
    n.box=pcd.get_oriented_bounding_box()
    n.box.color=[1,0,0]
    
    n.center=[n.box.center[0],n.box.center[1],np.mean(points[:,2])]
    n.dimensions=[n.box.extent[1],n.box.extent[2],zmax-zmin]
    rotation_matrix=copy.deepcopy(n.box.R) #! can these angles be negative?
    r =  Rotation.from_matrix(np.asarray(rotation_matrix))
    n.rotations = r.as_euler("zyx",degrees=True)

## EXPORT

las file

In [50]:
#merge point clouds
joined_pcd=gmu.join_geometries([n.resource for n  in pcdNodes])

In [51]:
#obtain labels
labels_segmentation=[]
labels_objects=[]

for i,n in enumerate(pcdNodes):
    length=len(np.asarray(n.resource.points))
    labels_segmentation.extend(list(np.full(length,n.class_id)))
    labels_objects.extend(list(np.full(length,n.object_id)))  

In [52]:
labels_segmentation=np.array([labels_segmentation])
labels_objects=np.array([labels_objects])

In [53]:
# create new las
header = laspy.LasHeader(point_format=3, version="1.2")
las = laspy.LasData(header)
las.xyz=np.asarray(joined_pcd.points)
las.red=(np.asarray(joined_pcd.colors)[:,0]*65535).astype(np.uint16)
las.green=(np.asarray(joined_pcd.colors)[:,1]*65535).astype(np.uint16)
las.blue=(np.asarray(joined_pcd.colors)[:,2]*65535).astype(np.uint16)

gmu.las_add_extra_dimensions(las,(labels_segmentation,labels_objects),['classes','objects'],['uint8','uint8'])
print(list(las.point_format.dimension_names))

['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'gps_time', 'red', 'green', 'blue', 'classes', 'objects']


In [54]:
las.write(pcd_output_path)
print(pcd_output_path)

c:\Users\Maarten\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\beton_labels.laz


json with object labels

In [55]:
#declare json
#json contains class_id, material_id, object_id, box and ifcGuid + status if present
json_data = {
        "filename": ut.get_filename(objects_output_path),
        "objects": []
    }

In [57]:
#fill json
for n in nodes:
    obj = {
            "name": n.name,
            "class_id":n.class_id,
            "object_id":n.object_id,
            "centroid": {
                "x": n.center[0],
                "y": n.center[1],
                "z": n.center[2]
            },
            "dimensions": {
                "length": n.dimensions[0],
                "width": n.dimensions[1],
                "height": n.dimensions[2]
            },
            "rotations": {
                "x": 0,
                "y": 0,
                "z": n.rotations[0]
            }
            }
    json_data["objects"].append(obj)

In [58]:
#write this information to the 3D detection json
with open(objects_output_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)
print("JSON data written to file:", objects_output_path)

JSON data written to file: c:\Users\Maarten\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\beton_objects.json


## FINISH

In [59]:
now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

print("runtime: --- %s seconds ---" % (time.time() - start_time))


Current Time = 12:03:50
runtime: --- 380.65604066848755 seconds ---
